### Import the library

In [1]:
import torch
import pickle

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

c:\Users\Admin\anaconda3\Lib\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "


In [2]:
from base_function import *

### Data Preparation

In [3]:
pickle_path = r'A:\Code\pso_rl\data\parameter_dataset.pkl'
with open(pickle_path, 'rb') as f:
    all_functions_by_dim = pickle.load(f)

In [4]:
train_data_combined = []
test_data_combined = []

for d, func_dict in all_functions_by_dim.items():
    for func_type, func_list in func_dict.items():
        if func_type == "schwefel" or func_type == "zakharov" :
            continue
        
        train_funcs, test_funcs = train_test_split(func_list, test_size=0.3, random_state=42)

        train_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in train_funcs])
        test_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in test_funcs])

print('split done')


split done


In [5]:
from collections import Counter

def count_functions_by_dim(data_combined, dim_value):
    # Tạo một Counter để đếm số lượng hàm của từng func_type với dim = dim_value
    func_count_by_type = Counter()
    
    # Duyệt qua từng phần tử trong list
    for item in data_combined:
        # Kiểm tra nếu dim bằng giá trị mong muốn
        if item['dim'] == dim_value:
            func_count_by_type[item['func_type']] += 1
    
    return func_count_by_type

# Đếm số hàm có dim = 1 trong train_data_combined
train_func_count = count_functions_by_dim(train_data_combined, 1)
print("Số lượng hàm có dim = 1 trong train_data_combined:", train_func_count)

# Đếm số hàm có dim = 1 trong test_data_combined
test_func_count = count_functions_by_dim(test_data_combined, 1)
print("Số lượng hàm có dim = 1 trong test_data_combined:", test_func_count)


Số lượng hàm có dim = 1 trong train_data_combined: Counter({'ackley': 140, 'griewank': 140, 'largeman': 140, 'levy': 140, 'styblinski_tang': 140})
Số lượng hàm có dim = 1 trong test_data_combined: Counter({'ackley': 60, 'griewank': 60, 'largeman': 60, 'levy': 60, 'styblinski_tang': 60})


In [6]:
splited_data = {'train': train_data_combined, 'test': test_data_combined}
splited_path = r'A:\Code\pso_rl\splited_data_no.pkl'
with open(pickle_path, 'wb') as f:
    pass
with open(splited_path, 'wb') as f:
    pickle.dump(splited_data, f)

print(f"All functions split into combined train and test sets and saved to '{splited_path}'")

All functions split into combined train and test sets and saved to 'A:\Code\pso_rl\splited_data_no.pkl'


In [7]:
def test_functions():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dims = [2, 5, 10]  # Different dimensions to test
    num_samples = 4  # Number of samples per test
    num_functions = 3  # Number of random functions per type

    for func_name in ['ackley', 'griewank', 'largeman', 'levy', 'styblinski_tang']:
        print(f"\nTesting {func_name} function")

        # Generate random test cases for each dimension
        for dim in dims:
            # Generate random input
            x = torch.rand((num_samples, dim), device=device) * 10 - 5  # Random values in [-5, 5]

            # Generate random parameters
            # Generate random parameters
            generator = getattr(globals()["".join([part.capitalize() for part in func_name.split('_')])], 'generate_functions')

            params_list = generator(num_functions, [dim])

            for params in params_list[dim]:
                # Get the function instance
                func = Function.get_function(func_name, x, params, device)

                # Evaluate the function
                result = func.evaluate_function()

                # Print results
                print(f"Input: {x}\nParameters: {params}\nOutput: {result}\n")

# Run the test script
test_functions()


Testing ackley function
Input: tensor([[-3.9401,  0.5907],
        [ 3.2077, -0.0619],
        [-3.6233,  1.6076],
        [-3.0691, -4.2298]], device='cuda:0')
Parameters: [18.74098576746199, 0.15932756981654728, 5.887616798020661]
Output: tensor([[8.9738],
        [5.7735],
        [9.0600],
        [8.7358]], device='cuda:0')

Input: tensor([[-3.9401,  0.5907],
        [ 3.2077, -0.0619],
        [-3.6233,  1.6076],
        [-3.0691, -4.2298]], device='cuda:0')
Parameters: [16.606082423147637, 0.24977857741245837, 4.807894682956034]
Output: tensor([[10.0878],
        [ 8.9035],
        [ 9.9367],
        [11.9462]], device='cuda:0')

Input: tensor([[-3.9401,  0.5907],
        [ 3.2077, -0.0619],
        [-3.6233,  1.6076],
        [-3.0691, -4.2298]], device='cuda:0')
Parameters: [16.893012478117896, 0.16466621882964963, 4.962876836941366]
Output: tensor([[8.0911],
        [6.9964],
        [7.6647],
        [9.9294]], device='cuda:0')

Input: tensor([[-3.1535, -1.2473, -3.3192, -2